## Feature Engineering

#### import data

In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
df = pd.read_csv('../merged_df/merged_df.csv', sep=",", index_col=0)

Only keep lines that mention the stock 

In [77]:
df.head()

,date,Stock,Open,Low,Close,Volume,7_day_avg,30_day_avg,60_day_avg,Price_Change_Pct,Article Length,article_sentiment
0,2015-11-23,AMZN,34.138500,33.506001,33.949501,87702000.0,34.701857,33.766227,33.766227,1.576783,973.0,2.0
1,2015-11-23,AAPL,29.932501,29.334999,29.437500,129930000.0,32.228643,33.646061,33.646061,-1.299248,973.0,2.0
2,2016-01-28,AMZN,31.903000,29.877501,31.767500,280304000.0,30.081143,33.238644,33.238644,8.914032,384.0,5.0
3,2016-02-04,MSFT,52.810001,51.369999,52.000000,46987100.0,33.553286,33.898727,33.898727,-0.306748,793.0,2.0
4,2016-02-09,AMZN,24.919001,23.700001,24.103500,141558000.0,33.085929,34.059894,34.059894,-1.235404,378.0,2.0


In [68]:


# Mapping stock names to CEO and company-related keywords
stock_ceo_company_mapping = {

  "AAPL": ["Tim Cook", "Apple", "iphone", "mac", "AAPL", "iPad", "iMac", "MacBook", "Apple Watch", "Apple TV", "AirPods", "iCloud", "Apple Music", "Apple Pay", "Apple Arcade", "App Store", "Steve Jobs", "Jony Ive"],
  "AMZN": ["Jeff Bezos", "Amazon", "Jassy", "Prime", "aws", "AMZN", "Amazon Prime", "Kindle", "Echo", "Alexa", "Fire Tablet", "AmazonBasics", "Whole Foods", "Amazon Music", "Audible", "Andy Jassy"],
  "GOOG": ["Sundar Pichai", "Google", "Alphabet", "Pichai", "Android", "AdSense", "GOOG", "Google Search", "Google Chrome", "Google Pixel", "Google Home", "Google Nest", "YouTube", "Google Drive", "Google Maps", "Google Play", "Google Ads", "Sergey Brin", "Larry Page"],
  "MSFT": ["Satya Nadella", "Nadella", "Azure", "Windows", "Microsoft", "MSFT", "Xbox", "Office", "Surface", "Microsoft Teams", "Microsoft Edge", "OneDrive", "Skype", "LinkedIn", "Outlook", "Bill Gates", "Steve Ballmer"],
  "TSLA": ["Elon", "Musk", "Tesla", "Model S", "CyberTruck", "TSLA", "Model 3", "Model X", "Model Y", "Roadster", "Powerwall", "Supercharger Network", "Tesla Energy", "JB Straubel"]
}


# Function to check if the text contains mentions of the stock's related keywords
def mentions_stock(text, stock_name):
    """
    Returns True if the text mentions any keywords associated with the stock.
    """
    keywords = stock_ceo_company_mapping.get(stock_name, [])
    return any(keyword.lower() in text.lower() for keyword in keywords)

# Filter rows where the "text" column mentions the stock on the same line
df = df[df.apply(lambda row: mentions_stock(row["text"], row["Stock"]), axis=1)]

# Display the shape of the filtered dataframe and the first few rows
df.shape, df.head()


((4906, 18),
                                                  url  \
 3  https://www.cnbc.com/2015/02/24/apples-record-...   
 5  https://www.cnbc.com/2015/02/25/dockers-rise-f...   
 6  https://www.cnbc.com/2015/02/25/dockers-rise-f...   
 7  https://www.cnbc.com/2015/02/25/dockers-rise-f...   
 8  https://www.cnbc.com/2015/02/25/dockers-rise-f...   
 
                                             title        date       author  \
 3          Apple’s record rally depends on Obama?  2015-02-24  John Melloy   
 5  Docker's rise from sleeper to open source king  2015-02-25     Ari Levy   
 6  Docker's rise from sleeper to open source king  2015-02-25     Ari Levy   
 7  Docker's rise from sleeper to open source king  2015-02-25     Ari Levy   
 8  Docker's rise from sleeper to open source king  2015-02-25     Ari Levy   
 
                                          description  \
 3  The size of Apple's capital return program wil...   
 5  Docker spent 2014 partnering with the world's ...

#### 7 day moving average

In [69]:
df['7_day_avg'] = df['Close'].rolling(window=7).mean()
df['30_day_avg'] = df['Close'].rolling(window=30).mean()
df['60_day_avg'] = df['Close'].rolling(window=30).mean()
df = df.dropna()
df.reset_index(inplace=True, drop=True)

C:\Users\ethan\AppData\Local\Temp\ipykernel_19980\2363880513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['7_day_avg'] = df['Close'].rolling(window=7).mean()
C:\Users\ethan\AppData\Local\Temp\ipykernel_19980\2363880513.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['30_day_avg'] = df['Close'].rolling(window=30).mean()
C:\Users\ethan\AppData\Local\Temp\ipykernel_19980\2363880513.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [70]:
df.columns

Index(['url', 'title', 'date', 'author', 'description', 'text',
       'Article Length', 'sentiment', 'sentiment_label', 'Stock', 'Open',
       'Low', 'Close', 'Adjusted_Close', 'Volume', 'Month', 'Price_Change_Pct',
       'Adjusted_Close_Change', '7_day_avg', '30_day_avg', '60_day_avg'],
      dtype='object')

In [71]:
df =df[[ 'date', 'Stock', 'Open', 'Low', 'Close', 'Volume',"7_day_avg","30_day_avg","60_day_avg", 'Price_Change_Pct','Article Length', 'sentiment']]
df.rename(columns={'sentiment': 'article_sentiment'}, inplace=True)

df.reset_index(drop=True, inplace=True)

In [72]:
df.head()

,date,Stock,Open,Low,Close,Volume,7_day_avg,30_day_avg,60_day_avg,Price_Change_Pct,Article Length,article_sentiment
0,2015-11-23,AMZN,34.138500,33.506001,33.949501,87702000.0,34.701857,33.766227,33.766227,1.576783,973.0,2.0
1,2015-11-23,AAPL,29.932501,29.334999,29.437500,129930000.0,32.228643,33.646061,33.646061,-1.299248,973.0,2.0
2,2016-01-28,AMZN,31.903000,29.877501,31.767500,280304000.0,30.081143,33.238644,33.238644,8.914032,384.0,5.0
3,2016-02-04,MSFT,52.810001,51.369999,52.000000,46987100.0,33.553286,33.898727,33.898727,-0.306748,793.0,2.0
4,2016-02-09,AMZN,24.919001,23.700001,24.103500,141558000.0,33.085929,34.059894,34.059894,-1.235404,378.0,2.0


In [73]:
df.to_csv('data/df_fe.csv')

In [53]:
df.head()

,date,Stock,Open,Low,Close,Volume,Price_Change_Pct,Article Length,article_sentiment,7_day_avg,30_day_avg,60_day_avg
0,2015-11-23,AMZN,34.138500,33.506001,33.949501,87702000.0,1.576783,973.0,2.0,34.701857,33.766227,33.766227
1,2015-11-23,AAPL,29.932501,29.334999,29.437500,129930000.0,-1.299248,973.0,2.0,32.228643,33.646061,33.646061
2,2016-01-28,AMZN,31.903000,29.877501,31.767500,280304000.0,8.914032,384.0,5.0,30.081143,33.238644,33.238644
3,2016-02-04,MSFT,52.810001,51.369999,52.000000,46987100.0,-0.306748,793.0,2.0,33.553286,33.898727,33.898727
4,2016-02-09,AMZN,24.919001,23.700001,24.103500,141558000.0,-1.235404,378.0,2.0,33.085929,34.059894,34.059894


In [51]:
df.describe()

,Open,Low,Close,Volume,Price_Change_Pct,Article Length,article_sentiment,7_day_avg,30_day_avg,60_day_avg
count,4906.000000,4906.000000,4906.000000,4.906000e+03,4906.000000,4906.000000,4906.000000,4900.000000,4877.000000,4877.000000
mean,173.587017,169.412269,171.607418,6.300889e+07,0.224299,794.774766,2.568080,171.651155,171.837794,171.837794
std,88.572323,86.740397,87.695801,4.726615e+07,2.324978,625.822052,1.367585,58.719032,55.030021,55.030021
min,15.496000,14.768000,15.463333,8.567800e+06,-21.062824,153.000000,1.000000,25.324381,32.545739,32.545739
25%,113.730003,110.022499,112.187500,2.829962e+07,-0.888947,422.000000,1.000000,144.622874,151.654665,151.654665
50%,151.891663,147.851662,149.997498,5.122040e+07,0.202414,649.000000,2.000000,177.497141,179.238176,179.238176
75%,235.820007,228.362495,232.899994,8.162080e+07,1.415841,954.000000,4.000000,211.564389,211.151667,211.151667
max,403.250000,382.000000,402.863342,4.265100e+08,14.216888,5369.000000,5.000000,331.650473,274.681603,274.681603


## Determining which faetures to use 

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4877 entries, 0 to 4876
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               4877 non-null   object 
 1   Stock              4877 non-null   object 
 2   Open               4877 non-null   float64
 3   Low                4877 non-null   float64
 4   Close              4877 non-null   float64
 5   Volume             4877 non-null   float64
 6   7_day_avg          4877 non-null   float64
 7   30_day_avg         4877 non-null   float64
 8   60_day_avg         4877 non-null   float64
 9   Price_Change_Pct   4877 non-null   float64
 10  Article Length     4877 non-null   float64
 11  article_sentiment  4877 non-null   float64
dtypes: float64(10), object(2)
memory usage: 457.3+ KB


In [74]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression


# Correlation Analysis
correlation_matrix = df.corr()  # Calculate the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')  # Visualize with a heatmap
plt.show()  # Display the heatmap to identify potential features

# Feature Importance with Random Forest
target = 'Price_Change_Pct'  # The target variable
features = [col for col in df.columns if col != target]  # All columns except the target

# Train a Random Forest to get feature importance
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(df[features], df[target])

# Get feature importance scores
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': rf.feature_importances_  # Get the importance scores
})

# Sort by importance to identify the most important features
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

print(feature_importance)  # Display the feature importance scores


ValueError: could not convert string to float: '2015-11-23'